In [11]:
#使用者版 0711_SLE_Classification&DAI(based on SLE_2-2)
#此版特色: 
# 1.在問答時，titer若出現非正浮點數，則請使用者重填；condition問答時若出現非Y或N的字元，將回到loop上個問題，讓填入者在填一次(而非資料處理時請他重填) 
# 2.更加善用dict的"參照"功能(注意：dict用於標籤簡單、特定關鍵字時) WAY1:同樣先彙整問答時的資料，再做後續的答案與分數轉換
# 3.試著將前置作業(dict維護)與後續資訊處理與運算分為兩部分，方便於未來診斷標準更新時，只須修改前面的dict即可。
# 4.當輸入B或F時，讓使用者知道自己正跳至別題檢查，並可更正答案(v)(*希望呈現出目前答案給使用者參考?)；同時新增No information的項目

##前置作業(資料建構，以2-D Array方式儲存)
# create domain array 
import numpy as np
SLE_domain = np.array([['Constitutional'], ['Hematologic'], ['Neuropsychiatric'], ['Mucocutaneous'], ['Serosal'], ['Musculoskeletal'], ['Renal'], ['Antiphospholipid antibodies'], ['Complement_proteins'], ['SLE-specific antibodies']])
# condition nparray(10*4 = 7*4 + 3*4)
SLE_clinical_condition = np.array([['Fever', None, None, None], ['Leukopenia', 'Thrombocytopenia', 'Autoimmune hemolysis', None], ['Delirium', 'Psychosis', 'Seizure', None], ['Non-scarring alopecia', 'Oral ulcers', 'Subacute cutaneous OR discoid lupus', 'Acute cutaneous lupus'], ['Pleural or pericardial effusion', 'Acute pericarditis', None, None], ['Joint involvement',None, None, None], ['Proteinuria >0.5g/24h', 'Renal Biopsy Class II or Class V lupus nephritis', 'Renal Biopsy Class III or Class IV lupus nephritis', None]])
SLE_immunological_condition = np.array([['Anti-cardiolipin antibodies OR Anti-β2GP1 antibodes OR Lupus anticoagulant', None, None, None], ['Low C3 OR low C4', 'Low C3 AND low C4', None, None],['Anti-dsDNA antibody OR Anti-Smith antibody', None, None, None]])
SLE_condition = np.vstack((SLE_clinical_condition, SLE_immunological_condition))
# weight array
SLE_weight = np.array([[2, 0, 0, 0], [3, 4, 4, 0], [2, 3, 5, 0], [2, 2, 4, 6], [5, 6, 0, 0], [6, 0, 0, 0], [4, 8, 10, 0], [2, 0, 0, 0], [3, 4, 0, 0], [6, 0, 0, 0]])
# Ans array
SLE_ans = np.array(None, (object, [10, 4]))
# 製作問答loop_control_array(元素為各domain之condition個數) >>用於SLE各condition問答
SLE_loop_control = [None] * len(SLE_condition)
i = 0
while i < len(SLE_condition):
  j = 0
  while j < len(SLE_condition[i]):
    if SLE_condition[i, j] is None:
      SLE_loop_control[i] = j
      break
    else:
      SLE_loop_control[i] = len(SLE_condition[i])
    j += 1
  i += 1      

#------------------------------------------------------------------------------------------#

### 各流程建立function
#entry_criterion_防呆裝置
titer = None
def entry_criterion_防呆裝置():
  global titer
  while type(titer) != type(6.6) or titer <= 0:
    try:
      titer = float(input('ANA at a titer of 1:"?" on HEp-2 cells.(Please input a number of type"float".)'))
      if titer <= 0:
        print('You should input a positive number!')
        continue
    except ValueError:
      print('You got a ValueError.')  
    except:
      print('You got other exceptions.')    ##關於try、except:老師建議瓊舉的事情由電腦來做?(目前不知道其他方法處理)
  return 'finished'    
#問答前詳細說明判定標準
def 問答前詳細說明判定標準():
  while True:
    if input('''Here are something you should be informed of before the questionaire.    
*Note that all criteria are only to be counted if SLE is thought to be the most likely cause of the manifestation, 
and that occurrence of a criterion on at least one occasion is sufficient.!!!  
Please input "OK" if you have understood all the above.''') == "OK":
      return "OK"
      break
    else:
      print('Please read it again!')
      continue
# additive_criteria問答 (結合for loop與while loop 處理與使用者的問答以及答題紀錄，適用於clinical & immunological)     
# 以SLE_domain、SLE_condition、SLE_clinical_condition、SLE_ans、SLE_loop_control作為argument，考慮與SLEDAI_condition_QA合併
def ConditionCriteria_QA(domain_array, ConditionArray, ClinicalCondiitonArray, ans_array, LoopControlList): 
  i = 0
  while i < len(domain_array):
      if i == 0:
        print(f'\n*Clinacal Conditions Questionaire:')
      if i == len(ClinicalCondiitonArray): 
        print(f'\n*Immunological Conditions Questionaire:')
      print(f'\n{(i%len(ClinicalCondiitonArray))+1}. Questions about {domain_array[i, 0]} domain:') #字串格式化   
      j = 0
      while j < LoopControlList[i]:
          # if ConditionArray[i, j] is None:  >>>此想法用 LoopControlList 解決
          #   break or continue ?
          # else:
          print(f'({(i%len(ClinicalCondiitonArray))+1}-{j+1}) {ConditionArray[i, j]} ?')      #是否附註該condition的定義給使用者作確認(?)/back&forward(V)/預設no information(v)
          check = input( 'Please input "Y", "N" or "" to answer this question OR "B" or "F" to jump to another question for check.' )
          if check == "Y":
              ans_array[i, j] = check   #先用Y、N組成ans_array，之後再process為ans_Number_array(0, 1)，再與SLEDAI_weight相乘，得到ans_Weight_array
              j = j + 1
          elif check == "N":
              ans_array[i, j] = check
              j = j + 1
          elif check == "":
              j = j + 1  
          elif check == "B": 
              print(f"For check: Please input the corrected answer if necessary OR input \"B\" or \"F\" to jump to another question.")                   
              if j == 0:                   #當輸入B或F時，讓使用者知道自己正跳至別題檢查，並可更正答案(v)(*希望呈現出目前答案給使用者參考?)
                  if i != 0:
                      i -= 1 
                      j = LoopControlList[i] - 1 
                  else:
                      print("*This is the first question!(No questions above)")      
                  # i = i - 1 if i != 0 else i                
                  # j = len(clinical_Q[i]) - 1 if i != 0 else j     ##原版(可否簡潔?)      
              else:
                  j -= 1
          elif check == "F":
              print(f"For check: Please input the corrected answer if necessary OR input \"B\" or \"F\" to jump to another question.")
              if j == LoopControlList[i] - 1:
                  if i != len(domain_array) - 1:
                      i += 1
                      j = 0
                  else:
                      print("*This is the last question!(No questions below)")        
                  # i = i + 1 if i != len(clinical) - 1 else i
                  # j = 0 if i != len(clinical) - 1 else j      ##原版(可否簡潔?)
              else:
                  j += 1
          else:
              print("You just filled in a wrong key word(not \"Y\" or \"N\"), please be more careful and try again!")
              j = j  
      print("Current_Ans_Array: ", ans_array)      #每個domain問答結束後，呈現出目前答題狀況(v)(*惟最後一題無法更改>考慮新增一項for最後檢查?)        
      i += 1
  return ans_array
#---------------------------------------------------------------------------------------------------------------#
## 處理各domains的weights(適用於clinical & immunological)
## key idea: 先用Y、N組成ans_array(v)，之後再process為ans_Number_array(-1無此問題, 0無症狀或無資料, 1有症狀)，再與SLEDAI_weight相乘，得到ans_Weight_array
SLE_ans_Number_array = SLE_weight.copy()
#各domain之condition答案欄如出現No Information，則將此特例的位置index儲存至ans_No_Information(nested list)中>>>方便產出最終建議
SLE_ans_No_Information = []
for i in range(len(SLE_condition)):
  SLE_ans_No_Information.append([])    # 此list(SLE_ans_No_Information) 將用來放置各domain無資料的index
#process為ans_Number_array(-1, 0, 1)
def Change_YN_to_Number(Ans_Array, Loop_Control, AnsNumberArray, AnsNoInformation):
  # global SLE_ans_Number_array, SLE_ans_No_Information
  for i in range(len(Ans_Array)):
    for j in range(len(Ans_Array[i])):
      if Ans_Array[i, j] == "Y":
        AnsNumberArray[i, j] = 1
      elif Ans_Array[i, j] == "N":
        AnsNumberArray[i, j] = 0
      else:
        AnsNumberArray[i, j] = -1  #注意: 若有問題的那個位置No information，也將成為-1(算分要小心)  >> 已將其index儲存於變數ans_No_Information     
        if j < Loop_Control[i]:
          AnsNumberArray[i, j] = 0
          AnsNoInformation[i].append(j)
  return AnsNumberArray
#找出各domain最高分
def Find_Domain_Max(Ans_Weight_Array):
  Trueweight2 = []
  for i in Ans_Weight_Array:
    Trueweight2.append(max(i))
  return Trueweight2  
# 找到weight最高的domain  >>判定main determinant instead of major cause!!! 
# 或highest_score = max(Domain_weight)  
highest_score = 0 
def Find_Main_Detreminamt(DomainWeight, DomainArray, SLEClinacalCondition):
  global highest_score
  highest_score = max(DomainWeight)
  main_clinical_determinant = []
  main_immunological_determinant = []
  for i in range(len(DomainWeight)):
    if DomainWeight[i] == highest_score:
      if i < len(SLEClinacalCondition):
        main_clinical_determinant.append(f'{DomainArray[i, 0]} domain')
      else:
        main_immunological_determinant.append(f'{DomainArray[i, 0]} domain')          
  return main_clinical_determinant, main_immunological_determinant  

# SLE final diagnosis: SLE classification requires at least one clinical criterion and ≥10 points.
def SLE_Classification_Advice(total_score, DomainWeight, SLEClinacalCondition):
  no_clinical_criteria = 0
  for i in DomainWeight[0:len(SLEClinacalCondition)]:
    if i == 0:
      no_clinical_criteria += 1
  print(f'\n*Diagnostic result:')    
  if total_score >= 10:
    if no_clinical_criteria == len(SLEClinacalCondition):
      return f'No. This patient is probably not classified as SLE because he/she doesn\'t meet any single clinical criterion.' 
    else:
      return f'''Yes. This patient is probably classified as SLE.
Main determinants: (1)clinical: {main_clinical_determinant} ； (2)immunology: {main_immunological_determinant}
Each gets {highest_score} pts of total {total_score} pts and accounts for {highest_score/total_score * 100:.2f} % of total weights.'''
  else:
    return f'''No. This patient is probably not classified as SLE.
The total weights add up to only {total_score} pts, which doesn't reach the standard 10 pts for SLE diagnosis.'''

# No_Information_Suggestion
def No_Information_Suggestion(Ans_No_Information, DomainArray, ConditionArray):  # ConditionArray(SLE_condirion) ; DomainArray(SLE_domain)
  print(f'\nThe conditions with no information, if any, are as follows:') 
  list1 = []
  for i in range(len(Ans_No_Information)):
    if Ans_No_Information[i] == []:
      continue
    else:
      for j in Ans_No_Information[i]:
        print(f'{ConditionArray[i, j]} (in {DomainArray[i]} domain)')
        list1.append(f'{ConditionArray[i, j]} (in {DomainArray[i]} domain)')
  print(f'''\n*IF you have some questions with NO INFORMATION, those conditions are considered negative automatically by the App
and thus it is likely that the patient's disease severity is underestimated.   In this case, We suggest that you take further 
examinations to make up the dificiency and then you can get more accurate predictions from the App! Thanks!''')
  return list1    
##-------------------------------------------------------------我是分隔線-----------------------------------------------------------------##
#entry criterion
print('''Welcome to SLE diagnosis assistant!
*Instructions for users: This App serves only as a diagnosis assistant, which is not designed for diagnosis or treatment decisions.
Diagnosis of SLE remains the purview of an appropriately trained physician evaluating an individual patient.''')
print('\nHere comes entry criteron.')
entry_criterion_防呆裝置 = entry_criterion_防呆裝置()
if titer <= 80:   
  print('\nAdditive criteria are needed before final diagnosis.')
  print(f'''The following are some important conditions in clinical or immunological domains.
Please input correct key words for the App to run smoothly!
"Y" stands for "Yes"(i.e. a patient meets the condition); "N" for "No"(i.e. one doesn't meet the criteria); ""(None) for "no information" 
"B" or "F" respectively for "jump *Back or *Forward to another question" Thank you for cooperation!''')   
  Notification_before_additive_criteria_questionaire  = 問答前詳細說明判定標準()

  # SLE additive criteria各condition問答 (利用巢狀while loop 處理與使用者的問答以及答題紀錄，適用於clinical & immunological) 
  SLE_ans = ConditionCriteria_QA(SLE_domain, SLE_condition, SLE_clinical_condition, SLE_ans, SLE_loop_control)
  print(f'*SLEDAI_ans: {SLE_ans}')

  #Data-processing: SLE_ans 轉換為 ans_Number_array(-1, 0, 1)    
  SLE_ans_Number_array = Change_YN_to_Number(SLE_ans, SLE_loop_control, SLE_ans_Number_array, SLE_ans_No_Information)  
  # ans_Number_array與SLE_weight相乘，得到SLE_ans_Weight_array
  SLE_ans_Weight_array = np.multiply(SLE_ans_Number_array, SLE_weight)
  #找出各domain最高分
  Domain_weight = Find_Domain_Max(SLE_ans_Weight_array)
  # 找到weight最高的domain  >>判定main determinant instead of major cause!!! 
  main_clinical_determinant, main_immunological_determinant = Find_Main_Detreminamt(Domain_weight, SLE_domain, SLE_clinical_condition)
  main_clinical_determinant = tuple(main_clinical_determinant) 
  main_immunological_determinant = tuple(main_immunological_determinant)
  
  # 最終結果與呈現診斷建議
  total_score = sum(Domain_weight)
  SLE_classification_advice = SLE_Classification_Advice(total_score, Domain_weight, SLE_clinical_condition)
  print(SLE_classification_advice)
  # No_Information_Suggestion
  SLE_No_Information_Condition = No_Information_Suggestion(SLE_ans_No_Information, SLE_domain, SLE_condition) 
else:
  print('\nDiagnostic result:')
  print('This patient is probably not classified as SLE because he/she doesn\'t the entry criterion!')
print("\nReference: The result is based on the 2019 EULAR/ACR classification criteria of SLE.") 

##-------------------------------------------------------------我是分隔線-----------------------------------------------------------------##
##-------------------------------------------------------------我是分隔線-----------------------------------------------------------------##
#如果probably classified as SLE >>> 用SLEDAI進一步嚴重度分級
if SLE_classification_advice.startswith('Yes') and titer <= 80: # total_score >= 10:
  #盡量不要重複問(Fever, Thrombocytopenia, Psychosis, Seizure, Proteinuria)
  # create domain array 
  import numpy as np
  SLEDAI_domain = np.array([['Constitutional'], ['Hematologic'], ['Neuropsychiatric'], ['Mucocutaneous'], ['Serosal'], ['Musculoskeletal'], ['Renal'], ['Other'], ['Complement_proteins'], ['Other']])
  # condition nparray(10*6 = 8*6 + 2*6)
  SLEDAI_clinacal_condition = np.array([['Fever', None, None, None, None, None], ['Leukopenia', 'Thrombocytopenia', 'Hematuria', 'Pyuria', None, None], ['Psychosis', 'Seizure', 'Organic brain syndrome', 'Visual disturbance', 'Cranial nerve disorder', 'Lupus headache'], ['Alopecia', 'Rash', 'Mucosal ulcers', None, None, None], ['Pleurisy', 'Pericarditis', None, None, None, None], ['Arthritis', 'Myositis', None, None, None, None], ['Proteinuria', 'Urinary casts', None, None, None, None], ['CVA(Cerebrovascular accident)', 'Vasculitis', None, None, None, None]])
  SLEDAI_immunological_condition = ['Low complement(Low CH50, C3, or C4)', None, None, None, None, None], ['Increased DNA binding', None, None, None, None, None]
  SLEDAI_condition = np.vstack((SLEDAI_clinacal_condition, SLEDAI_immunological_condition))
  # weight array
  SLEDAI_weight = np.array([[1, 0, 0, 0, 0, 0], [1, 1, 4, 4, 0, 0], [8, 8, 8, 8, 8, 8], [2, 2, 2, 0, 0, 0], [2, 2, 0, 0, 0, 0], [4, 4, 0, 0, 0, 0], [4, 4, 0, 0, 0, 0], [8, 8, 0, 0, 0, 0], [2, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]])
  # Ans array
  SLEDAI_ans = np.array(None, (object, [10, 6]))
  # 製作問答SLEDAI_loop_control_array(元素為各domain之condition個數) >>用於SLEDAI各condition問答 SLEDAI_loop_control
  SLEDAI_loop_control = [None] * len(SLEDAI_condition)
  i = 0
  while i < len(SLEDAI_condition):
    j = 0
    while j < len(SLEDAI_condition[i]):
      if SLEDAI_condition[i, j] is None:
        SLEDAI_loop_control[i] = j
        break
      else:
        SLEDAI_loop_control[i] = len(SLEDAI_condition[i])
      j += 1
    i += 1      

### 各流程建立function
  #問答前詳細說明判定標準
  def SLEDAI_Notification_before_Questionaire():
    while True:
      if input('''Here are something you should be informed of before the questionaire.    
 *Note that you should carefully evaluate whether or not each descriptor in SLEDAI-2K sheet is present 
at the time of the visit or in the preceding 30 days, and then input the most appropriate answer!!!  
  Please input "OK" if you have understood all the above.''') == "OK":
        return "OK"
        break
      else:
        print('Please read it again!')
        continue
  ###問答func: 與SLE additive criterion共用
  ###處理各domains的weights(適用於clinical & immunological)
  ## key idea: 先用Y、N組成ans_array(v)，之後再process為ans_Number_array(-1無此問題, 0無症狀或無資料, 1有症狀)，再與SLEDAI_weight相乘，得到ans_Weight_array
  SLEDAI_ans_Number_array = SLEDAI_weight.copy()   #SLEDAI_ans_Number_array
  #各domain之condition答案欄如出現No Information，則將此特例的位置index儲存至SLEDAI_ans_No_Information(nested list)中>>>方便產出最終建議
  SLEDAI_ans_No_Information = []
  for i in range(len(SLEDAI_condition)):
    SLEDAI_ans_No_Information.append([])    # 此list(SLEDAI_ans_No_Information) 將用來放置各domain無資料的index

  #SLEDAI_Class(嚴重度分級)
  def SLEDAI_Class(SLEDAI_TotalScore):
    if SLEDAI_TotalScore == 0:
      return 'no activity'
    elif 1 <= SLEDAI_TotalScore <= 5:
      return 'mild activity'
    elif 6 <= SLEDAI_TotalScore <= 10:
      return 'moderate activity(Suggestion: greater than 50% probability of initiating therapy)'
    elif 11 <= SLEDAI_TotalScore <= 19: 
      return 'high activity(Suggestion: greater than 50% probability of initiating therapy)' 
    else:
      return 'very high activity(Suggestion: greater than 50% probability of initiating therapy)' 

  #--------------------------------------------------------------------------------#
  print(f'''--------------------------------------------------------------------------------------------------------------------
\nGiven that the patient probably suffers from SLE, we would like to further measure the disease activity baesd on "SLEDAI-2K(30 Days).2010".
Hopefully, we can help you understand the patient's current condition and even predict the outcome and prognosis.\n''')
  #問答前詳細說明判定標準
  SLEDAI_Notification_before_Questionaire()
  # SLEDAI各condition問答 (利用巢狀while loop 處理與使用者的問答以及答題紀錄，適用於clinical & immunological) 
  SLEDAI_ans = ConditionCriteria_QA(SLEDAI_domain, SLEDAI_condition, SLEDAI_clinacal_condition, SLEDAI_ans, SLEDAI_loop_control)
  print(f'*SLEDAI_ans: {SLEDAI_ans}')             #做到Clinical 與 immunological domain問題的整合，可任意修改其中答案

  #process為ans_Number_array(-1, 0, 1)    
  SLEDAI_ans_Number_array = Change_YN_to_Number(SLEDAI_ans, SLEDAI_loop_control, SLEDAI_ans_Number_array, SLEDAI_ans_No_Information)  
  # ans_Number_array與SLEDAI_weight相乘，得到SLEDAI_ans_Weight_array
  SLEDAI_ans_Weight_array = np.multiply(SLEDAI_ans_Number_array, SLEDAI_weight)
  #計算總分
  SLEDAI_total_score = np.sum(SLEDAI_ans_Weight_array)
  print(f'SLEDAI_total_score = {SLEDAI_total_score}pts')

  # SLEDAI分級與final suggestion
  SLEDAI_Class = SLEDAI_Class(SLEDAI_total_score)
  print(f'Judgement Result: This patient\'s SLE Disease Activity is classified as {SLEDAI_Class}.') 
  # No_Information_Suggestion
  SLEDAI_No_Information_Condition = No_Information_Suggestion(SLEDAI_ans_No_Information, SLEDAI_domain, SLEDAI_condition)
  print(f'''\nRegerence: The result is based on SLEDAI-2K(30 Days).2010.
  Thank you for your patronage!''')

Welcome to SLE diagnosis assistant!
*Instructions for users: This App serves only as a diagnosis assistant, which is not designed for diagnosis or treatment decisions.
Diagnosis of SLE remains the purview of an appropriately trained physician evaluating an individual patient.

Here comes entry criteron.
ANA at a titer of 1:"?" on HEp-2 cells.(Please input a number of type"float".)45.6

Additive criteria are needed before final diagnosis.
The following are some important conditions in clinical or immunological domains.
Please input correct key words for the App to run smoothly!
"Y" stands for "Yes"(i.e. a patient meets the condition); "N" for "No"(i.e. one doesn't meet the criteria); ""(None) for "no information" 
"B" or "F" respectively for "jump *Back or *Forward to another question" Thank you for cooperation!
Here are something you should be informed of before the questionaire.    
*Note that all criteria are only to be counted if SLE is thought to be the most likely cause of the man